In [1]:
#READING THE DATA SET 
df = spark.read.format("csv").option("header", "true").load("StudentsPerformance.csv")
df.show(10)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [3]:
#1.Print the schema
df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: string (nullable = true)
 |-- reading score: string (nullable = true)
 |-- writing score: string (nullable = true)



In [6]:
#2.Show distinct parental backgrounds of students

df.select("parental level of education").distinct().show()

+---------------------------+
|parental level of education|
+---------------------------+
|           some high school|
|         associate's degree|
|                high school|
|          bachelor's degree|
|            master's degree|
|               some college|
+---------------------------+



In [8]:
#3.Select the number of students depending on parents educational background
df.groupby("parental level of education").count().show()

+---------------------------+-----+
|parental level of education|count|
+---------------------------+-----+
|           some high school|  179|
|         associate's degree|  222|
|                high school|  196|
|          bachelor's degree|  118|
|            master's degree|   59|
|               some college|  226|
+---------------------------+-----+



In [9]:
#4.Take a count of students in each ethnic group
df.groupby("race/ethnicity").count().show()

+--------------+-----+
|race/ethnicity|count|
+--------------+-----+
|       group B|  190|
|       group C|  319|
|       group D|  262|
|       group A|   89|
|       group E|  140|
+--------------+-----+



In [15]:
#5.What are the top 3 marks in math

from pyspark.sql.functions import desc
df.select("math score").sort(desc("math score")).distinct().show(3)

+----------+
|math score|
+----------+
|        99|
|        98|
|        97|
+----------+
only showing top 3 rows



In [30]:
#6.Change the data type of math score, reading score and writing from string to double

from pyspark.sql.types import DoubleType
new_df=df.withColumn("math score",df["math score"].cast("double"))






In [31]:
new_df=new_df.withColumn("writing score",new_df["writing score"].cast("double"))

In [32]:
new_df=new_df.withColumn("reading score",new_df["reading score"].cast("double"))

In [33]:
new_df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: double (nullable = true)
 |-- reading score: double (nullable = true)
 |-- writing score: double (nullable = true)



In [44]:
#7. avg of all scores depending on the ethnicity 

from pyspark.sql import functions as F
new_df.groupBy("race/ethnicity").agg(F.avg('math score'), F.avg('reading score'),F.avg('writing score')).show()

+--------------+-----------------+------------------+------------------+
|race/ethnicity|  avg(math score)|avg(reading score)|avg(writing score)|
+--------------+-----------------+------------------+------------------+
|       group B|63.45263157894737| 67.35263157894737|              65.6|
|       group C|64.46394984326018| 69.10344827586206| 67.82758620689656|
|       group D|67.36259541984732| 70.03053435114504| 70.14503816793894|
|       group A|61.62921348314607| 64.67415730337079|62.674157303370784|
|       group E|73.82142857142857| 73.02857142857142| 71.40714285714286|
+--------------+-----------------+------------------+------------------+



In [51]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType
from pyspark.sql.functions import col,array_contains

In [57]:
#8. count the number of students who has completed test preparation course and the number of students who havent 
new_df.filter(col("test preparation course") == "completed").count()

358

In [58]:
#9. Number of students who havent completed the test
new_df.filter(col("test preparation course") == "none").count()

642

In [59]:
#10.Get the average score of the students for all subjects based on ethnicity
new_df.filter(col("test preparation course") == "completed").groupBy("race/ethnicity").agg(F.avg('math score'), F.avg('reading score'),F.avg('writing score')).show()

+--------------+-----------------+------------------+------------------+
|race/ethnicity|  avg(math score)|avg(reading score)|avg(writing score)|
+--------------+-----------------+------------------+------------------+
|       group B|67.19117647058823|  72.8529411764706| 71.88235294117646|
|       group C| 67.4957264957265| 73.65811965811966| 74.45299145299145|
|       group D|69.79268292682927| 74.14634146341463| 76.65853658536585|
|       group A|68.25806451612904| 71.03225806451613| 70.90322580645162|
|       group E|77.43333333333334| 76.66666666666667| 75.98333333333333|
+--------------+-----------------+------------------+------------------+

